In [3]:
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
import gspread
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
from apiclient import discovery
from apiclient.http import MediaFileUpload
# from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import googleapiclient.discovery

import sa_drive as drive

"sa_drive" module does all the interaction with Google Drive API Using Service Account to authenticate.

Note: A service Account acts as a "normal" Gsuite account, which means it has it's own "drive space", so you need to share the files or directories you want "service account" to see.

You can access Gsuite information in two ways:

service accounts (using SAs)
user accounts (using Oauth)
We use in this example Service Accounts because we want to use Cloud Function to be triggered without user input, which happens with Oauth (requires user browser authentication)

In [4]:
files = drive.get_sibs_files()
print(len(files))
print(files)

100
[{'id': '1O54W07mPZPYczdfJQeG7FXwupE0otY88', 'name': '201902070162.inp'}, {'id': '1VTGLRGjqjEAuoWruBkqh2xfpZrQtm8py', 'name': '201902050161.inp'}, {'id': '1kbUCO4nAZd3pbeG5jLEJ1z2JxMVcEo19', 'name': '201902230165.inp'}, {'id': '19LRfoX_9MR7j1_vYJTkO8F6FiRDO9QIw', 'name': '201902250166.inp'}, {'id': '1XuMEFcqGz3sLnee0Wyn_7fB9merx1pj9', 'name': '201902220164.inp'}, {'id': '1Nx5_ksyvktyDuSqzo4mANwteM9wFBRbq', 'name': '201902210163.inp'}, {'id': '1AvYWK2saprGgEdu9H8jIG4NJZU-Jqj6r', 'name': '201904010184.inp'}, {'id': '1K--x7Om76SDvYrDwOiWOrlIfKg22hmy3', 'name': '201903030167.inp'}, {'id': '1X8qGdltISxaJHPmCtplHyeWiEk7nDup8', 'name': '201903040168.inp'}, {'id': '14PGJEHyns_W1ke0Bj5OLEvb9WYmwRm7D', 'name': '201904020185.inp'}, {'id': '13eFoQgR9MDwU4b_G1BCuS-86S6eE81FD', 'name': '201903120170.inp'}, {'id': '1MIgDZod59hg5gBjJJPk1A8DIhVOK5CUv', 'name': '201904030186.inp'}, {'id': '1ngXpfyrp6dOEHsMxGRAhSNeTVgyPnthK', 'name': '201903080169.inp'}, {'id': '1hMyHm_DllFQOYQd3_H9up6OSmHIh5Bpy', 'n

In [5]:
# df = pd.Dataframe(list(files.items()),columns = ['id', 'name'])
print(type(files))
df = pd.Dataframe.from_records(files)
pd.head(df)

<class 'list'>


AttributeError: module 'pandas' has no attribute 'Dataframe'

In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('SA_sheet-write-access.json', scope)
client = gspread.authorize(creds)

sheet = client.open('Helpo-SIBS').sheet1

data = sheet.get_all_records()
sheet_df = pd.DataFrame(data)
sheet_df.head()

,test1,tes2,test3
0,1,2,3
1,11,22,33
2,111,222,333


In [23]:
widths = [1, 4, 7, 7, 9, 9, 5, 3, 2, 73]
header = ['reg', 'type', 'IDori', 'IDdest', 'lastDate', 'IDlast', 'entity', 'coin', 'VAT', 'filler']
file_info = pd.read_fwf('202002040340.inp', widths=widths, nrows=1, header=None) 
file_info.columns = header
file_info.head()


FileNotFoundError: [Errno 2] No such file or directory: '202002040340.inp'

In [253]:
widths = [1,2,1,4,8,12,1,10,5,15,10,5, 9, 9, 8, 1, 1, 12, 6]
header = ['reg', 'cod', 'PDD', 'IdSIBS', 'numSIBS','date', 'typeT', 'idT', 'idTrans', 'LocT', 'value', 'tarif', 'refMB', 'NIF', 'nFact', 'modCom', 'CodEmp', 'RespID', 'filler']
df = pd.read_fwf('2020020403dsdfd40.inp', widths=widths, skipfooter=1) 
df.columns = header


In [254]:
df['date']= pd.to_duu

In [258]:
df.head()

,reg,cod,PDD,IdSIBS,numSIBS,date,typeT,idT,idTrans,LocT,value,tarif,refMB,NIF,nFact,modCom,CodEmp,RespID,filler
0,1,4,2,460,363385,2020-02-04 08:37:00,P,0,0,MBWAY,200,0,111111111,240571215,41,0,0,NaN,NaN
1,1,4,2,461,663982,2020-02-04 15:53:00,P,0,0,MBWAY,1000,0,111111111,0,42,0,0,NaN,NaN


In [259]:
excelFrame = df[['date', 'NIF', 'value', 'LocT']]

In [262]:
excelFrame.head()

,date,NIF,value,LocT
0,2020-02-04 08:37:00,240571215,200,MBWAY
1,2020-02-04 15:53:00,0,1000,MBWAY


In [264]:
writer = ExcelWriter('SIBS_exporter.xlsx')
excelFrame.to_excel(writer,'test')
writer.save()